### Web Scraping Realoka.com
#### Daffa Kaisha Pratama Chandra - 55770
#### Scraping dilakukan pada tanggal 2 November 2023
---
##### Import library yang dibutuhkan & Persiapan BeautifulSoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
baseurl = "https://www.realoka.com/" #base url

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

---
### Rumah di Kabupaten Tangerang

In [3]:
url_tgr = []

for x in range(1,130): # page 1 sampai n
    k = requests.get(f'https://www.realoka.com/search.php?kw=&jid=1-Rumah&jsid=2&propinsi_id=36-Banten&kab_id=3603-Tangerang+Kab.&kt=&park=&smin=&smax=&minharga=&maxharga=&minhargasw=&maxhargasw=&furnish=&hrgper=&hadap=&akses=&page={x}').text #link page yang mau di scrape
    soup=BeautifulSoup(k,'html.parser')
    list_rumah = soup.find_all("div",{"class":"lisbox"}) # cari card tempat produknya

    for rumah in list_rumah: # cari link produk
        link = rumah.find("a", href=True)
        if link:
            url_tgr.append(baseurl + link['href'])

In [4]:
is_duplicate = len(url_tgr) != len(set(url_tgr))
is_duplicate

True

In [5]:
url_tgr = list(dict.fromkeys(url_tgr))

In [6]:
# Banyak data
print("Banyak data rumah di Kabupaten Tangerang: {} rumah".format(len(url_tgr)))

Banyak data rumah di Kabupaten Tangerang: 2310 rumah


In [7]:
data_tgr = [] # List untuk menampung data rumah yang telah di scraping
c = 0

for link in url_tgr:  # input data
    f = requests.get(link, headers=headers).text
    hun = BeautifulSoup(f, 'html.parser')

    # Harga
    try:
        price = hun.find("span", {"class": "detailpage-angka"}).text.replace('\n', "")
    except:
        price = None

    # Alamat
    try:
        location = hun.find("span", {"class": "post-address2"}).text.replace('\n', "")
    except:
        location = None

    # Deskripsi
    try:
        description = hun.find("div", {"class": "p-box-content padding-20 tx-rata iklan_desc word-wrap"}).text.replace('\n', "")
    except:
        description = None

    # Nama rumah
    try:
        name = hun.find("div", {"class": "post-title"}).text.replace('\n', "")
    except:
        name = None

    # Spesifikasi rumah 1
    try:
        spec_list = hun.find("ul", {"class": "rel-ls-none p-addtional-box tx-kiri"})
    except:
        spec_list = None

    spec_dict = {}

    # Ekstraksi spesifikasi rumah
    if spec_list:
        for spec in spec_list.find_all('li'):
            spec_name = spec.find('span', {'class': 'p-f-box-head'}).text
            spec_value = spec.find('span', {'class': 'p-f-box-info'}).text
            # Hapus kurung siku pada spec_value
            spec_value = spec_value.replace('[', '').replace(']', '')
            spec_dict[spec_name] = spec_value

    # Spesifikasi rumah 2
    try:
        spec_list2 = hun.find_all('li', {'class': 'col-md-4 col-sm-6b'})
    except:
        spec_list2 = None

    spec_dict2 = {}

    # Ekstraksi spesifikasi rumah 2
    if spec_list2:
        for specs in spec_list2:
            spec_values = specs.get_text().strip()
            spec_dict2[spec_values] = 1  # Set value = 1 jika terdapat spesifikasi yang tertera

    rmh_tgr = {
        "nama_rumah": name,
        "alamat": location,
        "harga": price,
        "deskripsi": description
    }

    rmh_tgr.update(spec_dict)
    rmh_tgr.update(spec_dict2)

    # Masukan data rumah ke list data_tgr
    data_tgr.append(rmh_tgr)
    c += 1
    print("completed", c)


completed 1
completed 2
completed 3
completed 4
completed 5
completed 6
completed 7
completed 8
completed 9
completed 10
completed 11
completed 12
completed 13
completed 14
completed 15
completed 16
completed 17
completed 18
completed 19
completed 20
completed 21
completed 22
completed 23
completed 24
completed 25
completed 26
completed 27
completed 28
completed 29
completed 30
completed 31
completed 32
completed 33
completed 34
completed 35
completed 36
completed 37
completed 38
completed 39
completed 40
completed 41
completed 42
completed 43
completed 44
completed 45
completed 46
completed 47
completed 48
completed 49
completed 50
completed 51
completed 52
completed 53
completed 54
completed 55
completed 56
completed 57
completed 58
completed 59
completed 60
completed 61
completed 62
completed 63
completed 64
completed 65
completed 66
completed 67
completed 68
completed 69
completed 70
completed 71
completed 72
completed 73
completed 74
completed 75
completed 76
completed 77
complete

In [8]:
# Convert spesifikasi rumah1 ke dataframe
df_spec = pd.DataFrame([spec_dict])
df_spec

,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,Kondisi,Kelengkapan,Jumlah Lantai,Menghadap,Parkir
0,3 tahun yang lalu,60 m2,36 m2,2,1,SHM - Sertifikat Hak Milik,Baru,Kosongan,1,Timur,1 mobil


In [9]:
# Convert spesifikasi rumah2 ke dataframe
df_specs = pd.DataFrame([spec_dict2])
df_specs

,Kolam Renang,AC,One Gate,Pengamanan
0,1,1,1,1


In [10]:
# Gabungkan spesifikasi rumah 1 dengan spesifikasi rumah 2
specs = pd.concat([df_spec, df_specs], axis=1)
specs

,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,Kondisi,Kelengkapan,Jumlah Lantai,Menghadap,Parkir,Kolam Renang,AC,One Gate,Pengamanan
0,3 tahun yang lalu,60 m2,36 m2,2,1,SHM - Sertifikat Hak Milik,Baru,Kosongan,1,Timur,1 mobil,1,1,1,1


In [11]:
# Covert data rumah ke dataframe
df_tgr = pd.DataFrame(data_tgr)

# Gabungkan data rumah dengan data spesifikasi rumah
df_kab = pd.concat([df_tgr, specs], axis=1)
df_kab.head()

,nama_rumah,alamat,harga,deskripsi,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,...,Sertifikat,Kondisi,Kelengkapan,Jumlah Lantai,Menghadap,Parkir,Kolam Renang,AC,One Gate,Pengamanan
0,Rumahku Matagara Tangerang dekat Pusat Pemerin...,"Jalan Raya Puspem Tigaraksa, Tangerang Kab",320.400.000,Deskripsi dan SpesifikasiCLUSTER RUMAHKU MATAG...,9 jam yang lalu,30 m2,60 m2,2,1,HGB - Hak Guna Bangunan,...,SHM - Sertifikat Hak Milik,Baru,Kosongan,1,Timur,1 mobil,1.0,1.0,1.0,1.0
1,"Cluster Terbaru Puri, Cluster Maleo, Banyak Pr...","Puri Jaya Pasar Kemis, Tangerang Kab",500.000.000,Deskripsi dan SpesifikasiPuri Jaya Pasar Kemis...,15 jam yang lalu,60 m2,46 m2,2,2,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Cluster Maleo Puri Jaya Pasar Kemis, DP Bisa 0","Puri Jaya Pasar Kemis, Tangerang Kab",500.000.000,Deskripsi dan SpesifikasiHunian cluster Maleo ...,16 jam yang lalu,60 m2,46 m2,2,2,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"New Cluster Puri Jaya, Cluster Maleo, dekat Ko...","Puri Jaya Pasar Kemis Nur Sodik, Tangerang Kab",440.000.000,Deskripsi dan SpesifikasiPuri Jaya adalah PERU...,6 hari yang lalu,60 m2,31 m2,2,1,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cluster Tanamera Residence dekat Bandara Soett...,"Jalan Gatot Subroto Tanah Merah Sepatan, Tang...",297.000.000,Deskripsi dan SpesifikasiCLUSTER TANAMERA RESI...,1 minggu yang lalu,60 m2,30 m2,2,1,SHM - Sertifikat Hak Milik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Tambahkan variabel url dan wilayah
df_kab['Wilayah'] = 'Kabupaten Tangerang'
df_kab['url'] = url_tgr

In [13]:
df_kab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   nama_rumah     2310 non-null   object 
 1   alamat         2310 non-null   object 
 2   harga          2310 non-null   object 
 3   deskripsi      2310 non-null   object 
 4   Tayang Sejak   2310 non-null   object 
 5   Luas Tanah     2310 non-null   object 
 6   Luas Bangunan  2310 non-null   object 
 7   Kamar Tidur    2310 non-null   object 
 8   Kamar Mandi    2310 non-null   object 
 9   Sertifikat     2310 non-null   object 
 10  Kondisi        2310 non-null   object 
 11  Kelengkapan    2310 non-null   object 
 12  Jumlah Lantai  2310 non-null   object 
 13  Menghadap      2310 non-null   object 
 14  Daya Listrik   1666 non-null   object 
 15  Parkir         2310 non-null   object 
 16  AC             533 non-null    float64
 17  Kitchen Set    399 non-null    float64
 18  Alat Dap

In [14]:
df_kab.to_csv("rumah_kab_tangerang.csv", index=False)